#### Configure demo environment

We encourage the use of Python virtual envvironments as it allows users to configure a python environment for a specific purpose without introducing package conflicts.

We believe most python versions should work smoothly. Our Python version is 3.11.3. All the required libraries to run this demo are commonly used in CV & ML research area. We provide the list of required libraries in './demo_requirement.txt'. You may execute the following commands in the terminal to run the demo successfully. Alternatively, we also provide the output of this demo in a pdf file as './demo_output.pdf'. This file can be use for quick output check, comparisons, etc.  

1. Create a virtual environment named "demo_env"

```
python -m venv demo_env
```

2. Activate demo_env on Windows

```
./demo_env/Scripts/activate
```

OR on Linux/Unix/MacOS

```
source ./demo_env/Scripts/activate
```

3. Install PyTorch (if GPU is available) following the command on https://pytorch.org/. In our case, we have CUDA 11.8 so the command is

```
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
```
To check the CUDA version, run "nvidia-smi" in a Terminal.

4. Install packages

```
pip install -r demo_requirement.txt
```

5. Make sure this notebook runs the correct kernel

In [ ]:
from torchvision import models
import sys, random, os
sys.path.append('./src/utils')
sys.path.append('./src/visualize/')
import src.step3_test_on_SPASL_v1.MODULE_test as mod_test
import src.visualize.MODULE_visualize as mod_vis
import src.utils.MODULE_utils as mod_utils
import pandas as pd

## Experiment 1: Curve fitting on top-5 prediction probabilities

To quantify the prediction confidence of a model on an image, we first map the top-5 predictin probabilities ($p_i$, where $p_i \in (0, 1)$) to five points ($(x_i, p_i)$, where $x_i \in \{0.3, 0.9, 1.5, 2.1, 2.7\}$) in quadrant I. Then, we fit a curve on these five points. The best one we found is a scaled exponential curve: $f(x) = C \cdot e^{-\lambda x}$. 

Below, we demonstrate the process of fitting the curve on top-5 prediction probabilities for a model on an image. Predictions provided here are from AlexNet and MaxVit on $500$ n01440764 images. We also compare the fitted curves on the same image based on two models' predictions.   


In [ ]:
# The path of the IW table of AlexNet on 500 n01440764 images
model_name1 = 'AlexNet'
IW_path1 = f'./demo/exp_materials/{model_name1}_n01440764_after_AA.csv'
IW_df1 = pd.read_csv(IW_path1)

# Choose a random image
img_idx = random.randint(0, len(IW_df1)-1)

# Extract 5 predictions from AlexNet on it
prediction_probabilities1 = list(IW_df1.iloc[img_idx, 1:6])

# Plot the curve
mod_vis.plot_curve_and_points(prediction_probabilities1, f'AlexNet on {IW_df1.iloc[img_idx, 0]}')


In [ ]:
# We compare the predictin proabilities of a more recent model, MaxVit, with AlexNet on the same 500 images
model_name2 = 'MaxVit'
IW_path2 = f'./demo/exp_materials/{model_name2}_n01440764_after_AA.csv'
IW_df2 = pd.read_csv(IW_path2)

# Choose a random image
img_idx = random.randint(0, 499)

# Extract 5 prediction probabilities of AlexNet and MaxVit on it
prediction_probabilities1 = list(IW_df1.iloc[img_idx, 1:6]) # AlexNet's prediction probabilities
prediction_probabilities2 = list(IW_df2.iloc[img_idx, 1:6]) # MaxVit's prediction probabilities

# Ensure their predictions are about the same image
assert IW_df1.iloc[img_idx, 0] == IW_df2.iloc[img_idx, 0], 'Please make sure the images are the same.'
image_name = IW_df1.iloc[img_idx, 0]

# Plot the fitted curve and compare the parameters
mod_vis.compare_curves_on_the_same_image(prediction_probabilities1, model_name1, prediction_probabilities2, model_name2, image_name)

## Experiment 2: $\lambda$ distribution of a model on a class of images

After gathering $5$ predictions and probabilities of a model on each image in a class, we determine the $\lambda$ of the fitted exponential curve and $\text{correct\_idx}$. A larger value of $\lambda$ indicates higher confidence. $\text{correct\_idx}=-1$ indicates the correct prediction is not among the $5$ predictions. $\text{correct\_idx}=0$ suggests that the prediction (whose rank is $1$) with the highest probabilty is the correct one. The other possible values of $\text{correct\_idx}$ are $1,2,3,4$. They are categorized as ``non rank 1'' predictions. 

In this experiment, we show the distribution of $\lambda$ of AlexNet and MaxVit. Clear $\lambda$ distribution difference can be observed between two given models.

In [ ]:
# We provide two IW tables: AlexNet and MaxVit on 500 n014407864 images after applying adversarial distortion
# IW table location: './demo/exp_materials/AlexNet(or MaxVit)_n01440764_after_AA.csv'

model_name = 'AlexNet'
class_name = 'n01440764'
csv_path = f'./demo/exp_materials/{model_name}_{class_name}_after_AA.csv'
mod_vis.plot_single_prediction_and_lambda(csv_path, model_name = model_name, class_name = class_name, show = True, save_path = None)

In [ ]:
model_name = 'MaxVit'
csv_path = f'./demo/exp_materials/{model_name}_{class_name}_after_AA.csv'
mod_vis.plot_single_prediction_and_lambda(csv_path, model_name = model_name, class_name = class_name, show = True, save_path = None)

After extensive experiments and observations, we provide an empirical $\lambda$ value to quantify the boundary between confident and not confident predictions of a model on an image. In this way, we can determine the prediction pattern of a model on an image into one of the six identified patterns: Type-I/II/III/IV and Type-I/II-NRO.

|                 | **I** | **I-NRO** | **II** | **II-NRO** | **III** | **IV** |
|:---------------:|:-----:|:---------:|:------:|:----------:|:-------:|:------:|
|  **$\lambda$**  | $>4$  |    $>4$   |   $<4$ |    $<4$    |  $<4$   |  $>4$  |
| **correct_idx** |  $0$  |    $>0$   |   $0$  |    $>0$    |   $-1$  |  $-1$  |

We also define that if an image tends to cause Type-x prediction pattern for a lot of models, this image is called a Type-x image. The $500$ n014407864 images listed in the table are Type-I images, determined by cross searching. In other words, they are the easiest $500$ out of $1300$ images originally provided by ImageNet-1K in this class. 

## Experiment 3: Cross search

In [ ]:
# We test all ICAC models on each image and compute the lambda value of the fitted curve, and the correct_idx
# They are usually stored in difference csv files for a clean look

# Location of SPASL-general's IW_table of lambda values. This benchmark variant has an ICAC size of 80 
# Total # colums = 81
IW_table_lambda = pd.read_csv('./demo/exp_materials/IW_table_general_lambda_n01440764.csv')

# We display the first 5 rows
IW_table_lambda.head(5)

In [ ]:
# If a SPASL benchmark variant has a smaller ICAC, the IW_table construction will be faster.

# Location of SPASL-vit's IW_table of lambda values. This benchmark variant has an ICAC size of 12 
# Total # colums = 13
IW_table_lambda_vit = pd.read_csv('./demo/exp_materials/IW_table_vit_lambda_n01440764.csv')

# We display the first 5 rows
IW_table_lambda_vit.head(5)

In [ ]:
# Back to SPASL-general. Its IW_table of correct_idx also has 81 columns 
IW_table_correct_id = pd.read_csv('./demo/exp_materials/IW_table_general_correct_idx_n01440764.csv')

# We display the first 5 rows
IW_table_correct_id.head(5)

In [ ]:
# We calculate some statistics of lambda values, and count the number of correct (rank-1, non-rank-1), and wrong predictions.
# They are stored in the IW_summary table
IW_table_summary_path = './demo/exp_materials/IW_table_general_summary_n01440764.csv'
IW_table_summary = pd.read_csv(IW_table_summary_path)

# We display the first 5 rows
IW_table_summary.head(5)

In [ ]:
# Cross search on conditions of "R1_Count" (primary) and "lambda_Q3" for 500 Type-I images
Type_I_500_n01440764 = mod_utils.cross_search_for_top_n_images(IW_table_summary_path, 500, 'I')

# Display the top-10 images with the most R1_Count and lambda_Q3, thus, most models produce Type-I prediction patterns on them.
Type_I_500_n01440764.head(10)

In [ ]:
# Display the last 10 images of these 500 Type-I images
# It proves that they are also easy: 68 out of 80 can still provide a Type-I prediction pattern
Type_I_500_n01440764.tail(10)

### Comments

1. The image-wise (IW) table construction is based on the test results of ICAC models. A smaller ICAC can boost the IW table construction speed, hence preferred.

2. For each of $1\,000$ classes in ImageNet-1K, we generate a corresponding IW table, which initially gathers top-5 predictions and probabilites, and later $\lambda$ and $\text{correct\_idx}$ information. According to this IW table, we determine $100$ Type-IV images, and $500$ Type-I images by cross search. Type-IV images are the most difficult ones in each class, and they are included as the SuperHard (SH) component. For the rest four components, we apply modifications to $500$ easiest images and cross search for $100$ most difficulty modified images that fool most ICAC models. These images are included as the corresponding SPASP component.

3. Reason for constructing SH based on ImageNet-1K training set, rather than test set, is that all the pretrained models have been trained on the training set for a lot of times. In other words, they have seen all the training images for many times. If most of them still cannot provide a correct prediction, the only reason is that the image is very difficult. As human beings, we may not be able to explain why some images fool so many models: they look fine to us. This explains why the interpretability studies are important. And our benchmark will promote such studies. Obtaining the final SPASL\_v1, we do observe some mislabeled images that pretends to fool a lot of models. We overlook this mislabeling issue for now.

4. Reason for constructing the other four components based on easiest images in each class is that we want to make sure that an image converted from Type-I to Type-IV is only because the modification, not the image being difficult at the first place. In this way, we can evaluate the effectiveness of a modification, or attack, by studying the rates of such Type-I to Type-IV conversion. Reversely, we can evaluate restoration methods.  

The modifications are detailed explained in the paper and we will skip the experiment of modified example generation.

## Experiment 4: Test a model on SPASL\_demo 

SPASL\_demo is a tiny subsets ($10$ classes, $\approx 50$ MB) of SPASL\_v1 ($1\,000$ classes, $\approx 6$ GB), only for demo purpose. This experiment demonstrates how to test a model and how the results are generated and unfolded. 

Test results of a model include: 

1. Top-1 and top-5 accuracies on five SPASL components with the corresponding SPASL score calculated ('./demo/result/result_on_whole_SPASL/{model}/{model}_top1(and 5)_acc.csv').

2. An entry of the tested model performance added to the local SPASL test history ('./demo/result/result_on_whole_SPASL/history.csv(and .json)').

3. IW tables (Optional, provided in the experiment at './demo/result/IW_table/{model}'). 

4. Radar charts (Optional, provided in the experiment at './demo/result/IW_table/{model}').

When assessing a model using the authentic SPASL\_v1, test results will be provided in the same folder hierarchy.


In [ ]:
# First, we test ResNet-18 on SPASL_demo.

model = models.resnet18(weights='DEFAULT')
model_name = 'resnet18'

# The location of SPASL_demo
SPASL_dir = './demo/SPASL_demo'
# Location to store all test results
result_mother_folder = './demo/result'
# Test the current model for the first time
mod_test.test_on_whole_SPASL_benchmark(model, model_name, SPASL_dir, result_mother_folder, bm_name = 'SPASL_demo', draw_radar_graph = True, progress_bar = True)

In [ ]:
# When evaluating the same model in the future, test results will be directly retrieved from the local test histroy. 
mod_test.test_on_whole_SPASL_benchmark(model, model_name, SPASL_dir, result_mother_folder, bm_name = 'SPASL_demo', draw_radar_graph = True, progress_bar = True)

In [ ]:
# We encourage you to try as many models as you would like to.
# We provide several here.
# More models can be found at  "https://pytorch.org/vision/stable/models.html"

#************************************************
#MaxVit
#model = models.maxvit_t(weights='DEFAULT')
#model_name = 'maxvit_t'

#ShuffleNet_v2_x0_5
model = models.shufflenet_v2_x0_5(weights='DEFAULT')
model_name = 'shufflenet_v2_x0_5'

#VitB-16 (330 MB, download may take longer)
#model = models.vit_b_16(weights='DEFAULT')
#model_name = 'vit_b_16'
#************************************************

mod_test.test_on_whole_SPASL_benchmark(model, model_name, SPASL_dir, result_mother_folder, bm_name = 'SPASL_demo', draw_radar_graph = True, progress_bar = True)

***FAQ*** 

**Q1.** How to solve the error ".../Activate.ps1 cannot be loaded because running scripts is disabled on this system." when trying to activate a virtual env?

**A1.** Enter the command "Set-ExecutionPolicy -Scope Process -ExecutionPolicy Bypass", then activate the virtual env. 